# 1. CNN with MNIST

In [3]:
# MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
# CNN Model (3 conv layers and 2 Fully Connected Layers)
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5 # for dropout
        # L1 ImgIn shape=(7, 28, 28, 1)
        #    Conv    ->  (7, 28, 28, 32)
        #    Pool    ->  (7, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # L2 ImgIn shape=(7, 14, 14, 32)
        #    Conv    ->  (7, 14 ,14, 64)
        #    Pool    ->  (7, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # L3 ImgIn shape=(7, 7, 7, 64)
        #    Conv    ->  (7, 7, 7, 128)
        #    Pool    ->  (7, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        ) 
        # L4 FC 4×4×128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p= 1 - self.keep_prob)
        )
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out
        

In [16]:
# instantiate CNN model
model = CNN()
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (layer4): Sequential(
    (0): Linear(in_features=2048, out_features=625, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


In [17]:
# model confirm
value = torch.Tensor(1, 1, 28, 28)
print( model(value).shape )

torch.Size([1, 10])


In [13]:
# define cost & optimizer
criterion = torch.nn.CrossEntropyLoss() # softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # image is already size of (28×28), no reshape
        # label is not one-hot encoded
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print(' [Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
 [Epoch:    1] cost = 0.190949321
 [Epoch:    2] cost = 0.0508460291
 [Epoch:    3] cost = 0.0367996693
 [Epoch:    4] cost = 0.0282784421
 [Epoch:    5] cost = 0.0231997892
 [Epoch:    6] cost = 0.0200016983
 [Epoch:    7] cost = 0.0155272251
 [Epoch:    8] cost = 0.0136048226
 [Epoch:    9] cost = 0.01281974
 [Epoch:   10] cost = 0.0112234605
 [Epoch:   11] cost = 0.0101432884
 [Epoch:   12] cost = 0.00828325097
 [Epoch:   13] cost = 0.00898345187
 [Epoch:   14] cost = 0.00800768752
 [Epoch:   15] cost = 0.00771812722
Learning Finished!


In [15]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9692999720573425


# 2. Visdom

In [23]:
# import
import visdom

In [25]:
vis = visdom.Visdom()

Setting up a new session...


'window_3ae5b513a03e68'

In [26]:
vis.text("Hello, World!", env="main") # env: 나중에 창을 끌 때 사용할 수 있음

'window_3ae5b54201d474'

In [27]:
vis.image(torch.randn(3, 200, 200))

'window_3ae5b54ca6af78'

In [28]:
vis.images(torch.Tensor(3, 3, 28, 28))

'window_3ae5b55305af7a'

In [33]:
data = mnist_train.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_3ae5b5bded8508'

In [34]:
# data loader로 여러 개 한 번에 확인 가능
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([100, 1, 28, 28])


In [38]:
# Line Plot
Y_data = torch.randn(5) # x값 없으면 0부터 1 사이의 값 생성됨
plt1 = vis.line(Y=Y_data)

X_data = torch.Tensor([1,2,3,4,5])
plt2 = vis.line(Y=Y_data, X=X_data)

In [39]:
# close the window
vis.close(env="main")

''

Connection to remote host was lost.
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 61] Connection refused
on_close() takes 1 positional argument but 3 were given


## Multiple Line on Single Windows

In [40]:
num = torch.Tensor(list(range(0,10)))
print(num.shape)
num = num.view(-1,1)
print(num.shape)
num = torch.cat((num,num),dim=1) # Y와 똑같은 shape이어야 함 (10,2 -> 2개 plot)
print(num.shape)

plt = vis.line(Y=torch.randn(10,2), X = num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


## Line info

In [41]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [43]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title='Test', legend=['1번'], showlegend=True)) # 범례 직접 설정

In [44]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번', '2번'], showlegend=True)) # 2개 동시에 plot

## Function for update line

In [46]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')

In [49]:
plt = vis.line(Y=torch.Tensor(1).zero_()) # plot 창 띄움

for i in range(500):
    loss = torch.randn(1) + i # 값이 점점 올라갈 것!
    loss_tracker(plt, loss, torch.Tensor([i]))

## MNIST with visdom

In [61]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init as init
import torch.optim as optim

### import visdom

In [51]:
import visdom
vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

### Define loss_tracker

In [52]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
             )

### Device setup

In [53]:
# if gpu avialable
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


### parameters

In [54]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

### Load MNIST dataset

In [55]:
# MNIST datset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform=transform.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [57]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

### Build CNN model

In [59]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [60]:
model = CNN().to(device)

value = (torch.Tensor(1, 1, 28, 28)).to(device)
print( (model(value)).shape)

torch.Size([1, 10])


In [62]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Make plot

In [63]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),
                    opts=dict(title='loss_tracker',
                              legend=['loss'], showlegend=True))

### Train with loss_tracker

In [65]:
# Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch: {} / {}      cost = {}'.format(
        epoch+1, training_epochs, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

Epoch: 1 / 15   \ cost = 0.12016356736421585
Epoch: 2 / 15   \ cost = 0.040132440626621246
Epoch: 3 / 15   \ cost = 0.028882423415780067
Epoch: 4 / 15   \ cost = 0.0223945751786232
Epoch: 5 / 15   \ cost = 0.018104173243045807
Epoch: 6 / 15   \ cost = 0.014409651048481464
Epoch: 7 / 15   \ cost = 0.011822243221104145
Epoch: 8 / 15   \ cost = 0.011364157311618328
Epoch: 9 / 15   \ cost = 0.009236768819391727
Epoch: 10 / 15   \ cost = 0.009085382334887981
Epoch: 11 / 15   \ cost = 0.007849275134503841
Epoch: 12 / 15   \ cost = 0.009674713015556335
Epoch: 13 / 15   \ cost = 0.006667201407253742
Epoch: 14 / 15   \ cost = 0.00499905738979578
Epoch: 15 / 15   \ cost = 0.007920607924461365
Learning Finished!


### Test

In [66]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9846000075340271
